In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split
import bitsandbytes as bnb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from huggingface_hub import login

/home/rifat/anaconda3/envs/secretbench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import set_seed

set_seed(69420)

In [ ]:
df1 = pd.read_csv("dataset/secretbench_merged_filecontents_1.csv")
df1=df1[df1['Label']==0]
df1 = df1.sample(frac=1, random_state=69420).reset_index(drop=True).head(15000)

In [ ]:
df2 = pd.read_csv("dataset/secretbench_merged_filecontents_2.csv")
df2=df2[df2['Label']==0]
df2 = df2.sample(frac=1, random_state=69420).reset_index(drop=True).head(15000)

In [5]:
df3 = pd.read_csv("dataset/secretbench_label_1_samples.csv")
df3 = df3.sample(frac=1, random_state=69420).reset_index(drop=True).head(15000)

In [6]:
# Combine negatives
df_neg = pd.concat([df1, df2]).sample(frac=1, random_state=69420).reset_index(drop=True)  # 30,000 negatives
df_pos = df3  # 15,000 positives

del df1
del df2

In [7]:


# Splitting train, eval, test
train_pos = df_pos.iloc[:12000]  # 12k positives for balanced train
train_neg = df_neg.iloc[:12000]  # 12k negatives for balanced train
train_set = pd.concat([train_pos, train_neg]).sample(frac=1, random_state=69420).reset_index(drop=True)

imbalanced_train_pos = df_pos.iloc[:3750]  # 3.75k positives for imbalanced train
imbalanced_train_neg = df_neg.iloc[:20250]  # 20.25k negatives for imbalanced train
imbalanced_train_set = pd.concat([imbalanced_train_pos, imbalanced_train_neg]).sample(frac=1, random_state=69420).reset_index(drop=True)

eval_pos = df_pos.iloc[12000:13500]  # 1.5k positives for eval
eval_neg = df_neg.iloc[20250:21750]  # 1.5k negatives for eval
eval_set = pd.concat([eval_pos, eval_neg]).sample(frac=1, random_state=69420).reset_index(drop=True)

test_pos = df_pos.iloc[13500:15000]  # 1.5k positives for test
test_neg = df_neg.iloc[21750:23250]  # 1.5k negatives for test
test_set = pd.concat([test_pos, test_neg]).sample(frac=1, random_state=69420).reset_index(drop=True)

# Save datasets
train_set.to_csv("final_dataset/train_set.csv", index=False)
imbalanced_train_set.to_csv("final_dataset/imbalanced_train_set.csv", index=False)
eval_set.to_csv("final_dataset/eval_set.csv", index=False)
test_set.to_csv("test_set.csv", index=False)

print("final_dataset/Datasets saved successfully!")

final_dataset/Datasets saved successfully!
